## This notebook demonstrates how to perform bulk inference of [DeepSeek R1 Zero](https://github.com/deepseek-ai/DeepSeek-R1) on the [Tracto.ai](https://tracto.ai/) platform.

In [1]:
import yt.wrapper as yt
import uuid

In [2]:
yt.config["pickling"]["dynamic_libraries"]["enable_auto_collection"] = False
yt.config["pickling"]["ignore_system_modules"] = True
yt.config["pickling"]["safe_stream_mode"] = False

In [3]:
working_dir = f"//tmp/examples/tractorun-mnist_{uuid.uuid4()}"
yt.create("map_node", working_dir, recursive=True)
print(working_dir)

//tmp/examples/tractorun-mnist_3b54cfef-739f-4204-a8d8-515e38ce9098


Prepare data for inference as a YTSaurus table.

In [5]:
from datasets import load_dataset

dataset = load_dataset("Rapidata/Other-Animals-10")

table_path = f"{working_dir}/questions"
yt.create("table", table_path, force=True)

questions = [
    {"question": f"Can {animal} fly?"}
    for animal in set(dataset["train"].features["label"].int2str(dataset["train"]["label"]))
]

yt.write_table(table_path, questions)

Run bulk inference of DeepSeek R1 Zero in two jobs.

In [7]:
from typing import Iterable
import logging
import sys

@yt.aggregator
def bulk_inference(records: Iterable[dict[str, str]]) -> dict[str, str]:
    from vllm import LLM, SamplingParams

    # yt job have to write all logs to stderr
    vllm_logger = logging.getLogger("vllm")
    vllm_logger.handlers.clear()
    vllm_logger.addHandler(logging.StreamHandler(sys.stderr))

    llm = LLM(model="deepseek-ai/DeepSeek-R1-Zero", tensor_parallel_size=8, trust_remote_code=True)
    sampling_params = SamplingParams(
        temperature=0.7,
        top_p=0.9,
        max_tokens=10000,
    )

    for record in records:
        conversations = [
            {
                "role": "user",
                "content": record["question"],
            },
        ]
        outputs = llm.chat(
            messages=conversations,
            sampling_params=sampling_params,
        )
        yield {
            "prompt": record["question"],
            "text": outputs[0].outputs[0].text,
        }

In [8]:
result_path = f"{working_dir}/result"

yt.run_map(
    bulk_inference,
    table_path,
    result_path,
    job_count=2,
    spec={
        "pool": "fifo",
        "pool_trees": ["gpu_h200"],
        "mapper": {
            "gpu_limit": 8,
            "memory_limit": 322122547200,
            "cpu_limit": 64,
        },
    },
)

2025-02-04 22:10:37,039	INFO	Operation started: https://playground.yt.nebius.yt/playground/operations/a021c270-36e42e9c-270703e8-38d89242/details


2025-02-04 22:10:37,056	INFO	( 0 min) operation a021c270-36e42e9c-270703e8-38d89242 starting


2025-02-04 22:10:37,588	INFO	( 0 min) operation a021c270-36e42e9c-270703e8-38d89242 initializing


2025-02-04 22:10:38,685	INFO	( 0 min) Unrecognized spec: {'enable_partitioned_data_balancing': false, 'mapper': {'title': 'bulk_inference'}}


2025-02-04 22:10:38,715	INFO	( 0 min) operation a021c270-36e42e9c-270703e8-38d89242: running=0     completed=0     pending=2     failed=0     aborted=0     lost=0     total=2     blocked=0    


...

2025-02-04 22:11:21,507	INFO	( 0 min) operation a021c270-36e42e9c-270703e8-38d89242: running=2     completed=0     pending=0     failed=0     aborted=1     lost=0     total=2     blocked=0    


2025-02-04 22:53:46,064	INFO	(43 min) operation a021c270-36e42e9c-270703e8-38d89242: running=1     completed=1     pending=0     failed=0     aborted=1     lost=0     total=2     blocked=0    


2025-02-04 22:54:01,162	INFO	(43 min) operation a021c270-36e42e9c-270703e8-38d89242 completing


2025-02-04 22:54:01,685	INFO	(43 min) operation a021c270-36e42e9c-270703e8-38d89242 completed


2025-02-04 22:54:01,727	INFO	(43 min) Alerts: {'low_cpu_usage': {'code': 1, 'message': "Average CPU usage of some of your job types is significantly lower than requested 'cpu_limit'. Consider decreasing cpu_limit in spec of your operation", 'attributes': {'pid': 1, 'tid': 11289248088709684183, 'thread': 'Controller:13', 'fid': 18446274782774647951, 'host': 'man0-0461.hw.nebius.yt', 'datetime': '2025-02-04T22:54:00.157894Z', 'trace_id': '5afb4819-4e17fb68-e61f393e-f64d2ace', 'span_id': 11470563213524108392}, 'inner_errors': [{'code': 1, 'message': 'Jobs of task "map" use 2.70% of requested cpu limit', 'attributes': {'pid': 1, 'tid': 11289248088709684183, 'thread': 'Controller:13', 'fid': 18446274782774647951, 'host': 'man0-0461.hw.nebius.yt', 'datetime': '2025-02-04T22:54:00.157874Z', 'trace_id': '5afb4819-4e17fb68-e61f393e-f64d2ace', 'span_id': 11470563213524108392, 'cpu_time': 8838941, 'cpu_limit': 64.0, 'exec_time': 5113090}}]}}


In [9]:
for record in yt.read_table(result_path):
    print(record)

{'prompt': 'Can snail fly?', 'text': '<think>\nTo answer the question "Can snail fly?", we need to understand what a snail is and what "flying" means.\n\nA snail is a type of mollusk characterized by a coiled shell (in most species) and a soft, unsegmented body. Snails are usually found on land (as in the case of land snails) or in water (as in the case of aquatic snails such as pond snails or sea snails). Snails move primarily by using a muscular foot that glides along a layer of mucus that they secrete, which makes their movement typically slow and ground-based.\n\n"Flying" refers to the act of moving through the air, typically by using wings. Most commonly, animals such as birds, insects, and bats are known for their ability to fly. \n\nBased on this understanding, snails do not have wings or any other anatomical features that would allow them to fly. Therefore, the answer to the question "Can snail fly?" is:\n\nNo, a snail cannot fly.\nThis reasoning is based on the biological char